### Kansas Data Cleaning Notebook
#### Data from Kansas State University Sorghum Experiments (Ashland Bottoms)

- please contact David LeBauer at dlebauer@email.arizona.edu or Emily Cain at ejcain@email.arizona.edu with any questions, or create an issue in this [repository](https://github.com/genophenoenvo/terraref-datasets)

In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import requests

#### Trait data queried and downloaded from betydb in `R` using this code:
```
library(traits)

options(betydb_url = "https://terraref.ncsa.illinois.edu/bety/",
        betydb_api_version = 'v1',
        betydb_key = 'secret_api_key_123456_abcde')
        
kansas <- betydb_query(sitename  = "~Bottoms",
                         limit     =  "none")
                      
write.csv(kansas, file = "kansas_experiments_2020-06-11.csv")
```

#### Weather data downloaded from [KSU Ashland Bottoms Weather Station](https://mesonet.k-state.edu/weather/historical/)

In [2]:
def download_csv(url, folder_name, file_name):
    response = requests.get(url)
    with open(os.path.join(folder_name, file_name), 'wb') as f:
        f.write(response.content)

In [3]:
def read_in_csv(folder_name, file_name):
    df = pd.read_csv(folder_name + '/' + file_name, low_memory=False)
    return df

In [4]:
def save_to_csv_without_timestamp(list_of_dfs, list_of_output_filenames):

    for i,j in zip(list_of_dfs, list_of_output_filenames):
        i.to_csv(j, index=False)

In [5]:
def save_to_csv_without_timestamp(list_of_dfs, list_of_output_filenames):

    for i,j in zip(list_of_dfs, list_of_output_filenames):
        i.to_csv(j, index=False)

In [6]:
folder_name = 'data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [7]:
ksu_url = 'https://de.cyverse.org/dl/d/3A0BFAB4-2A09-4E58-9350-8AE3927414BE/ksu_data_2020-06-11.csv'
ksu_input_filename = 'ksu_trait_data.csv'

In [8]:
download_csv(ksu_url, folder_name=folder_name, file_name=ksu_input_filename)

In [9]:
df_0 = read_in_csv(folder_name=folder_name, file_name=ksu_input_filename)
# print(df_0.shape)
# df_0.head()

### Slice for selected traits
- canopy height
- days & GDD to flowering

In [10]:
# traits available in this dataset

df_0.trait.unique()

array(['canopy_height', 'seedling_vigor', 'Sugar_content',
       'crude_protein', 'lodging_percent', 'ndf',
       'aboveground_biomass_moisture', 'crown_color',
       'aboveground_fresh_biomass_per_plot', 'emergence_score',
       'leaf_length', 'leaf_width', 'leaf_attachment_angle', 'adf',
       'stem_width', 'flowering_time'], dtype=object)

In [11]:
df_1 = df_0.loc[(df_0.trait == 'flowering_time') | (df_0.trait == 'canopy_height')]
# print(df_1.shape)
# df_1.head(3)

#### Drop Columns & convert `date` to datetime object
- convert `raw_date` to new datetime object
- new datetime object will be in `date` column

In [12]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id', 'treatment_id', 
                'scientificname', 'commonname', 'genus', 'species_id', 'cultivar_id', 'author', 
                'citation_year', 'time', 'month', 'year', 'n', 'statname', 'stat', 'notes', 'access_level', 
                'entity', 'view_url', 'edit_url', 'date', 'dateloc', 'city', 'treatment']

df_2 = df_1.drop(labels=cols_to_drop, axis=1)
# print(df_2.shape)
# df_2.tail()

#### Convert `raw_date` to datetime object

In [13]:
new_dates = pd.to_datetime(df_2.raw_date)

df_3 = df_2.copy()
df_3['date'] = new_dates

# print(df_2.shape[0])
# print(df_3.shape[0])

# df_3.head()

#### Extract `Range` and `Pass` values

In [14]:
df_4 = df_3.copy()

df_4['range'] = df_4['sitename'].str.extract("Range (\d+)").astype(int)
df_4['pass'] = df_4['sitename'].str.extract("Pass (\d+)").astype(int)

# df_4.sample(n=3)

#### A. Canopy Height

In [15]:
df_4.trait.value_counts()

canopy_height     4044
flowering_time     656
Name: trait, dtype: int64

In [16]:
ch_0 = df_4.loc[df_4.trait == 'canopy_height']
print(ch_0.shape)
# ch_0.head(3)

(4044, 13)


#### Drop Columns
- trait
- units
- raw date
- method name

In [17]:
cols_to_drop = ['raw_date', 'trait', 'units', 'method_name']

ch_1 = ch_0.drop(labels=cols_to_drop, axis=1)
# print(ch_1.shape)
# ch_1.head()

#### Reorder & Rename Columns

In [18]:
ch_2 = ch_1.rename({'mean': 'canopy_height_cm'}, axis=1)
# print(ch_2.shape)
# ch_2.tail()

#### Reorder Columns

In [19]:
new_col_order = ['date', 'sitename', 'range', 'pass', 'lat', 'lon', 'cultivar', 'trait_description', 'canopy_height_cm']

ch_3 = pd.DataFrame(data=ch_2, columns=new_col_order)
# print(ch_3.shape)
# ch_3.head()

#### Check for duplicates

In [20]:
# ch_3.duplicated().value_counts()

#### Drop Duplicates

In [21]:
ch_4 = ch_3.drop_duplicates(subset=['date', 'sitename', 'range', 'pass', 'cultivar', 'canopy_height_cm'], ignore_index=True)

# print(ch_4.shape)
# ch_4.head(3)

#### B. Days & Growing Degree Days (GDD) to Flowering
- Weather data in csv format downloaded from [KSU Weather Station](https://mesonet.k-state.edu/weather/historical/) in Ashland Bottoms
- Queried using season dates
- planting date: 2016-06-17
- harvest date: 2016-10-21

In [22]:
print(df_4.raw_date.min())
print(df_4.raw_date.max())

2016-07-15 00:00:00 -0500
2016-10-21 00:00:00 -0500


In [23]:
ksu_weather_url = 'https://de.cyverse.org/dl/d/64805E3B-0460-4AA1-8D8A-2D7246E05B35/ashland_bottoms_daily_weather_2016.csv'
ksu_weather_input_filename = 'ksu_weather_data.csv'

In [24]:
download_csv(ksu_weather_url, folder_name=folder_name, file_name=ksu_weather_input_filename)

In [25]:
weather_0 = read_in_csv(folder_name=folder_name, file_name=ksu_weather_input_filename)
print(weather_0.shape)
weather_0.head()

(129, 16)


,Unnamed: 0,Timestamp,Station,AirTemperature,AirTemperature.1,RelativeHumidity,Precipitation,WindSpeed2m,WindSpeed2m.1,SoilTemperature5cm,SoilTemperature5cm.1,SoilTemperature10cm,SoilTemperature10cm.1,SolarRadiation,ETo,ETo.1
0,NaN,NaN,NaN,max,min,avg,total,avg,max,max,min,max,min,total,grass,alfalfa
1,NaN,NaN,NaN,°C,°C,%,mm,m/s,m/s,°C,°C,°C,°C,MJ/m²,mm,mm
2,NaN,2016-06-17,Ashland Bottoms,37.8,24,66.6,0,2.3,9,27.5,25,25.7,24,30.4,7.96,9.92
3,NaN,2016-06-18,Ashland Bottoms,33.1,21.7,66.4,5.33,2.8,12.1,26.7,24.5,25.4,24.3,22.7,6.33,8.26
4,NaN,2016-06-19,Ashland Bottoms,35.3,21.9,62.5,0,2.9,7.4,26.7,24,25.2,23.7,29.3,7.7,9.9


#### Change column names and drop first two rows
- Add datetime column

In [26]:
weather_1 = weather_0.rename({'Station': 'city', 'AirTemperature': 'air_temp_max_C', 
                                                  'AirTemperature.1': 'air_temp_min_C', 'RelativeHumidity': 'avg_rh',
                                                  'Precipitation': 'precip_mm', 'WindSpeed2m': 'avg_wind_speed', 
                                                  'WindSpeed2m.1': 'max_wind_speed', 'SoilTemperature5cm': 'soil_temp_5cm_max',
                                                  'SoilTemperature5cm.1': 'soil_temp_5cm_min', 
                                                  'SoilTemperature10cm': 'soil_temp_10cm_max', 
                                                  'SoilTemperature10cm.1': 'soil_temp_10cm_min', 'SolarRadiation': 'solar_rad',
                                                  'ETo': 'eto_grass', 'ETo.1': 'eto_alfalfa'}, axis=1)
# print(weather_1.shape)
# weather_1.head()

In [27]:
# Drop first 2 rows

weather_2 = weather_1.iloc[2:]
# print(weather_2.shape)
# weather_2.head()

In [28]:
weather_3 = weather_2.copy()

datetimes = pd.to_datetime(weather_3['Timestamp'])
weather_3['date'] = datetimes

# print(weather_3.shape)
# weather_3.tail()

#### Drop Columns
- timestamp
- Unnamed: 0

In [29]:
cols_to_drop = ['Unnamed: 0', 'Timestamp']

weather_4 = weather_3.drop(labels=cols_to_drop, axis=1)
# print(weather_4.shape)
# weather_4.head()

#### Convert numeric columns 

In [30]:
cols_to_convert = ['air_temp_max_C', 'air_temp_min_C', 'avg_rh', 'precip_mm', 'avg_wind_speed', 'max_wind_speed', 
                   'soil_temp_5cm_max', 'soil_temp_5cm_min', 'soil_temp_10cm_max', 'soil_temp_10cm_min', 'solar_rad', 
                   'eto_grass', 'eto_alfalfa', ]

weather_5 = weather_4.copy()
weather_5[cols_to_convert] = weather_5[cols_to_convert].apply(pd.to_numeric)
# print(weather_5.shape)
# print(weather_5.dtypes)

#### Add `daily_gdd` column and check for negative values

In [31]:
weather_6 = weather_5.copy()
weather_6['daily_gdd'] = (((weather_6['air_temp_max_C'] + weather_6['air_temp_min_C'])) / 2) - 10
# print(weather_6.shape)
# weather_6.head()

In [32]:
# Check for negative values

weather_6.loc[weather_6.daily_gdd < 0]

,city,air_temp_max_C,air_temp_min_C,avg_rh,precip_mm,avg_wind_speed,max_wind_speed,soil_temp_5cm_max,soil_temp_5cm_min,soil_temp_10cm_max,soil_temp_10cm_min,solar_rad,eto_grass,eto_alfalfa,date,daily_gdd
119,Ashland Bottoms,14.2,4.8,70.5,0.0,3.1,8.7,16.8,14.0,17.0,15.3,16.1,2.33,3.23,2016-10-12,-0.50
120,Ashland Bottoms,16.2,0.1,74.7,0.0,1.1,5.3,14.4,12.1,15.3,13.8,15.3,1.89,2.44,2016-10-13,-1.85


In [33]:
weather_7 = weather_6.copy()

for k,v in weather_7.iteritems():
    
    if k == 'daily_gdd':
        v[v < 0] = 0

<ipython-input-33-83fe36c110b4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v[v < 0] = 0


In [34]:
# should return empty df now

weather_7.loc[weather_7.daily_gdd < 0]

,city,air_temp_max_C,air_temp_min_C,avg_rh,precip_mm,avg_wind_speed,max_wind_speed,soil_temp_5cm_max,soil_temp_5cm_min,soil_temp_10cm_max,soil_temp_10cm_min,solar_rad,eto_grass,eto_alfalfa,date,daily_gdd


In [35]:
# Add cumulative GDD, round to nearest integer

weather_8 = weather_7.copy()

weather_8['gdd'] = np.rint(np.cumsum(weather_8['daily_gdd']))
# print(weather_8.shape)
# weather_8.tail()

#### Drop `daily_gdd` column, add cumulative precipitation column

In [36]:
weather_9 = weather_8.drop(labels=['daily_gdd'], axis=1)
# print(weather_9.shape)
# print(weather_9.columns)

In [37]:
weather_10 = weather_9.copy()
weather_10['cum_precip_mm'] = np.round(np.cumsum(weather_10.precip_mm), 2)

# print(weather_10.shape)
# weather_10.head(3)

#### Write Ashland Bottoms Weather Interim Data to `.csv`

In [38]:
# timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
# output_filename = f'data/interim/ashland_bottoms_weather_2016_daily_{timestamp}.csv'.replace(':', '')

# weather_10.to_csv(output_filename, index=False)

#### Add GDD to Days to Flowering DataFrame
- slice trait data to only include `days_to_flowering`
- merge DataFrames on `date_of_flowering`

In [39]:
# df_4.trait.value_counts()

In [40]:
fl_0 = df_4.loc[df_4.trait == 'flowering_time']
# print(fl_0.shape)
# fl_0.head()

In [41]:
cols_to_drop = ['raw_date', 'trait', 'units', 'method_name']

fl_1 = fl_0.drop(labels=cols_to_drop, axis=1)
# print(fl_1.shape)
# fl_1.tail()

#### Rename trait column

In [42]:
fl_2 = fl_1.rename({'mean': 'days_to_flowering'}, axis=1)
# print(fl_2.shape)
# fl_2.sample(n=3, random_state=42)

#### Add `planting_date`
- 2016-06-17

In [43]:
day_of_planting = datetime.date(2016,6,17)
fl_3 = fl_2.copy()

fl_3['date_of_planting'] = day_of_planting
# print(fl_3.shape)
# fl_3.head()

#### Create timedelta using `flowering_time` values

In [44]:
timedelta_values = fl_3['days_to_flowering'].values
dates_of_flowering = []

for val in timedelta_values:
    
    date_of_flowering = day_of_planting + datetime.timedelta(days=val)
    dates_of_flowering.append(date_of_flowering)
    
# print(fl_3.shape[0])
# print(len(dates_of_flowering))

In [45]:
fl_4 = fl_3.copy()
fl_4['date_of_flowering'] = dates_of_flowering
# print(fl_4.shape)
# fl_4.sample(n=5, random_state=42)

#### Add GDD flowering DataFrame

In [46]:
weather_gdd = weather_10[['date', 'gdd']]
# print(weather_gdd.shape)
# weather_gdd.head()

In [47]:
fl_5 = fl_4.copy()
fl_5.date_of_flowering = pd.to_datetime(fl_5.date_of_flowering)
# fl_5.dtypes

In [48]:
fl_6 = fl_5.merge(weather_gdd, how='left', left_on='date_of_flowering', right_on=weather_gdd['date'])
# print(fl_6.shape)
# fl_6.head()

#### Check for duplicates

In [49]:
# fl_6.duplicated().value_counts()

In [50]:
fl_7 = fl_6.drop_duplicates(subset=['sitename', 'days_to_flowering', 'cultivar', 'date_of_flowering', 'gdd'], 
                           ignore_index=True)

# print(fl_7.shape)
# fl_7.tail()

#### Drop all date columns except `date_of_flowering`

In [51]:
date_cols_to_drop = ['date_x', 'date_of_planting', 'date_y']

fl_8 = fl_7.drop(labels=date_cols_to_drop, axis=1)
# print(fl_8.shape)
# fl_8.head()

#### Write flowering dataframe to `.csv`

In [54]:
list_of_dfs = [ch_4, fl_8]
list_of_output_filenames = ['ksu_canopy_height.csv', 'ksu_days_gdd_to_flowering.csv']

save_to_csv_without_timestamp(list_of_dfs=list_of_dfs, list_of_output_filenames=list_of_output_filenames)